<a href="https://colab.research.google.com/github/areias/bert_covid_sentiment/blob/main/bert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training

from https://github.com/digitalepidemiologylab/covid-twitter-bert/blob/c87912b409659f40018e839c4124be5ae2486713/run_finetune.py


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [ ]:
#!git clone -b master https://github.com/digitalepidemiologylab/covid-twitter-bert.git

Cloning into 'covid-twitter-bert'...
remote: Enumerating objects: 1660, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 1660 (delta 150), reused 140 (delta 88), pack-reused 1438
Receiving objects: 100% (1660/1660), 3.48 MiB | 14.79 MiB/s, done.
Resolving deltas: 100% (1050/1050), done.


In [3]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2 MB 4.1 kB/s 
     |████████████████████████████████| 2.9 MB 77.8 MB/s 
     |████████████████████████████████| 3.0 MB 61.9 MB/s 
     |████████████████████████████████| 454 kB 62.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalli

In [4]:
import tensorflow
print(tensorflow.__version__)

2.2.0


In [5]:
!pip install  tensorflow_addons==0.11.2

     |████████████████████████████████| 1.1 MB 9.9 MB/s 


In [6]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 7.1 MB/s 
     |████████████████████████████████| 3.3 MB 49.5 MB/s 
     |████████████████████████████████| 895 kB 63.8 MB/s 
     |████████████████████████████████| 596 kB 80.3 MB/s 
     |████████████████████████████████| 59 kB 8.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
import sys
sys.path.append('drive/MyDrive/ct-bert')
sys.path.append('drive/MyDrive/ct-bert/tensorflow_models')


In [8]:
from official.nlp.bert import bert_models
from official.utils.misc import distribution_utils
from official.nlp.bert import configs as bert_configs
from official.modeling import performance
from official.nlp.bert import input_pipeline
from official.utils.misc import keras_utils

In [9]:
import os
import datetime
import time
import argparse
import math
import logging
from logging.handlers import RotatingFileHandler


In [10]:
import tqdm
import json
import tensorflow as tf
from utils.misc import ArgParseDefault, save_to_json, add_bool_arg
from utils.finetune_helpers import Metrics
import utils.optimizer
from config import PRETRAINED_MODELS

In [11]:
import datetime
def get_run_name():
    # Use timestamp to generate a unique run name
    ts = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S_%f')
    run_name = f'run_{ts}'
    return run_name


In [ ]:
"""python run_finetune.py \
  --run_prefix $RUN_PREFIX \
  --bucket_name $BUCKET_NAME \
  --tpu_ip $TPU_IP \
  --model_class $MODEL_CLASS \
  --finetune_data ${FINETUNE_DATA}/${FINETUNE_DATASET} \
  --train_batch_size $TRAIN_BATCH_SIZE \
  --eval_batch_size $EVAL_BATCH_SIZE \
  --num_epochs $NUM_EPOCHS \
  --learning_rate $LR"""



'python run_finetune.py   --run_prefix $RUN_PREFIX   --bucket_name $BUCKET_NAME   --tpu_ip $TPU_IP   --model_class $MODEL_CLASS   --finetune_data ${FINETUNE_DATA}/${FINETUNE_DATASET}   --train_batch_size $TRAIN_BATCH_SIZE   --eval_batch_size $EVAL_BATCH_SIZE   --num_epochs $NUM_EPOCHS   --learning_rate $LR'

In [26]:
# args

RUN_PREFIX='testrun'                                  # Name your run
#BUCKET_NAME=                                        # Fill in your buckets name here (without the gs:// prefix)
#TPU_IP=XX.XX.XXX.X                                  # Fill in your TPUs IP here
FINETUNE_DATASET='crowdbreaks'                      # Your dataset name
FINETUNE_DATA= get_run_name()                         # Fill in dataset run name (e.g. run_2020-05-19_14-14-53_517063_test_run)
MODEL_CLASS='bert_large_uncased_wwm'
TRAIN_BATCH_SIZE=8 #32
EVAL_BATCH_SIZE=8
LR=2e-5
NUM_EPOCHS=1

In [27]:
from collections import namedtuple
arguments = namedtuple('arguments', ['run_prefix','model_class','finetune_data', 
                                     'train_batch_size', 'eval_batch_size','learning_rate',
                                     'limit_train_steps','limit_eval_steps','num_epochs',
                                     'warmup_proportion', 'init_checkpoint','validation_freq',
                                     'end_lr','optimizer_type', 'save_model',
                                     'early_stopping_epochs','time_history_log_steps'])

args = arguments(RUN_PREFIX,MODEL_CLASS,FINETUNE_DATA,
                 TRAIN_BATCH_SIZE,EVAL_BATCH_SIZE,LR,
                 None,None,1,
                 0.1, None,1,
                 0,'adamw',True,
                1,10)            
args 

arguments(run_prefix='testrun', model_class='bert_large_uncased_wwm', finetune_data='run_2021-11-24_14-12-42_144149', train_batch_size=8, eval_batch_size=8, learning_rate=2e-05, limit_train_steps=None, limit_eval_steps=None, num_epochs=1, warmup_proportion=0.1, init_checkpoint=None, validation_freq=1, end_lr=0, optimizer_type='adamw', save_model=True, early_stopping_epochs=1, time_history_log_steps=10)

In [28]:
def set_mixed_precision_policy(args):
    """Sets mix precision policy."""
    if args.dtype == 'fp16':
        policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16', loss_scale=loss_scale)
        tf.keras.mixed_precision.experimental.set_policy(policy)
    elif args.dtype == 'bf16':
        policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
        tf.keras.mixed_precision.experimental.set_policy(policy)
    elif args.dtype == 'fp32':
        tf.keras.mixed_precision.experimental.set_policy('float32')
    else:
        raise ValueError(f'Unknown dtype {args.dtype}')

In [41]:
#def run(args):
"""Train using the Keras/TF 2.0. Adapted from the tensorflow/models Github"""
# CONFIG
run_name = 'run_2021-11-24_07-56-06_744272_test_run'
#logger.info(f'*** Starting run {run_name} ***')
data_dir = 'drive/MyDrive/ct-bert/data/finetune/'+run_name+'/crowdbreaks'
output_dir = 'drive/MyDrive/ct-bert/data/finetune/'+run_name+'/crowdbreaks'

output_dir

'drive/MyDrive/ct-bert/data/finetune/run_2021-11-24_07-56-06_744272_test_run/crowdbreaks'

In [42]:
def get_model_config(config_path):
    config = bert_configs.BertConfig.from_json_file(config_path)
    return config

In [43]:
PRETRAINED_MODELS[args.model_class]['config']

'bert_config_large_uncased_wwm.json'

In [44]:
def get_model_config_path(args):
    try:
        config_path = PRETRAINED_MODELS[args.model_class]['config']
    except KeyError:
        raise ValueError(f'Could not find a pretrained model matching the model class {args.model_class}')
    return os.path.join('drive/MyDrive/ct-bert/configs/', config_path)

In [45]:
# Get configs
pretrained_model_config_path = get_model_config_path(args)
model_config = get_model_config(pretrained_model_config_path)

In [46]:
pretrained_model_config_path

'drive/MyDrive/ct-bert/configs/bert_config_large_uncased_wwm.json'

In [47]:
model_config

In [49]:
def get_input_meta_data(data_dir):
    with tf.io.gfile.GFile(data_dir+'/meta.json', 'rb') as reader:
        input_meta_data = json.loads(reader.read().decode('utf-8'))
    return input_meta_data


In [50]:
def get_label_mapping(data_dir):
    with tf.io.gfile.GFile(data_dir+'/label_mapping.json', 'rb') as reader:
        label_mapping = json.loads(reader.read().decode('utf-8'))
    label_mapping = dict(zip(range(len(label_mapping)), label_mapping))
    return label_mapping

In [51]:
# Meta data/label mapping
input_meta_data = get_input_meta_data(data_dir)
label_mapping = get_label_mapping(data_dir)
#logger.info(f'Loaded training data meta.json file: {input_meta_data}')

In [52]:
input_meta_data

{'eval_data_size': 680,
 'max_seq_length': 96,
 'num_labels': 3,
 'processor_type': 'text-classification',
 'task_type': 'bert_classification',
 'train_data_size': 2041}

In [53]:
label_mapping

{0: -1, 1: 0, 2: 1}

In [54]:
# Calculate steps, warmup steps and eval steps
train_data_size = input_meta_data['train_data_size']
num_labels = input_meta_data['num_labels']
max_seq_length = input_meta_data['max_seq_length']
if args.limit_train_steps is None:
    steps_per_epoch = int(train_data_size / args.train_batch_size)
else:
    steps_per_epoch = args.limit_train_steps
warmup_steps = int(args.num_epochs * train_data_size * args.warmup_proportion/ args.train_batch_size)
if args.limit_eval_steps is None:
    eval_steps = int(math.ceil(input_meta_data['eval_data_size'] / args.eval_batch_size))
else:
    eval_steps = args.limit_eval_steps

In [55]:
def get_model(args, model_config, steps_per_epoch, warmup_steps, num_labels, max_seq_length, is_hub_module=False):
    # Get classifier and core model (used to initialize from checkpoint)
    if args.init_checkpoint is None and PRETRAINED_MODELS[args.model_class]['is_tfhub_model']:
        # load pretrained model from TF-hub
        hub_module_url = f"https://tfhub.dev/{PRETRAINED_MODELS[args.model_class]['hub_url']}"
        hub_module_trainable = True
    else:
        hub_module_url = None
        hub_module_trainable = False
    classifier_model, core_model = bert_models.classifier_model(
            model_config,
            num_labels,
            max_seq_length,
            hub_module_url=hub_module_url,
            hub_module_trainable=hub_module_trainable)
    # Optimizer
    optimizer = utils.optimizer.create_optimizer(
            args.learning_rate,
            steps_per_epoch * args.num_epochs,
            warmup_steps,
            args.end_lr,
            args.optimizer_type)
    classifier_model.optimizer = configure_optimizer(
            optimizer,
            use_float16=False,
            use_graph_rewrite=False)
    return classifier_model, core_model

In [56]:
def configure_optimizer(optimizer, use_float16=False, use_graph_rewrite=False, loss_scale='dynamic'):
    """Configures optimizer object with performance options."""
    if use_float16:
        # Wraps optimizer with a LossScaleOptimizer. This is done automatically in compile() with the
        # "mixed_float16" policy, but since we do not call compile(), we must wrap the optimizer manually.
        optimizer = (tf.keras.mixed_precision.experimental.LossScaleOptimizer(optimizer, loss_scale=loss_scale))
    if use_graph_rewrite:
        # Note: the model dtype must be 'float32', which will ensure
        # tf.ckeras.mixed_precision and tf.train.experimental.enable_mixed_precision_graph_rewrite do not double up.
        optimizer = tf.train.experimental.enable_mixed_precision_graph_rewrite(optimizer)
    return optimizer

In [57]:
def get_loss_fn(num_classes):
    """Gets the classification loss function."""
    def classification_loss_fn(labels, logits):
        """Classification loss."""
        labels = tf.squeeze(labels)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        one_hot_labels = tf.one_hot(tf.cast(labels, dtype=tf.int32), depth=num_classes, dtype=tf.float32)
        per_example_loss = -tf.reduce_sum(tf.cast(one_hot_labels, dtype=tf.float32) * log_probs, axis=-1)
        return tf.reduce_mean(per_example_loss)
    return classification_loss_fn

In [58]:
# Get model
classifier_model, core_model = get_model(args, model_config, steps_per_epoch, warmup_steps, num_labels, max_seq_length)
optimizer = classifier_model.optimizer
loss_fn = get_loss_fn(num_labels)
#try:
#    if ',' in args.validation_freq:
#        validation_freq = args.validation_freq.split(',')
#        validation_freq = [int(v) for v in validation_freq]
#    else:
validation_freq = int(args.validation_freq)
#except:
#    raise ValueError(f'Invalid argument for validation_freq!')
#logger.info(f'Using a validation frequency of {validation_freq}')


In [59]:
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 96)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 96)]         0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 96)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [60]:
classifier_model.get_config()

{'input_layers': {'input_mask': ['input_mask', 0, 0],
  'input_type_ids': ['input_type_ids', 0, 0],
  'input_word_ids': ['input_word_ids', 0, 0]},
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 96),
    'dtype': 'int32',
    'name': 'input_word_ids',
    'ragged': False,
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_word_ids'},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 96),
    'dtype': 'int32',
    'name': 'input_mask',
    'ragged': False,
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_mask'},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 96),
    'dtype': 'int32',
    'name': 'input_type_ids',
    'ragged': False,
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_type_ids'},
  {'class_name': 'KerasLayer',
   'config': {'dtype': 'float32',
    'handle': 'https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/2',
    'name': 'keras_l

In [61]:
def get_metrics():
    return [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]


In [62]:
# Run keras compile
#logger.info(f'Compiling keras model...')
classifier_model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=get_metrics())
#logger.info(f'... done')

In [63]:
# Create all custom callbacks
summary_dir = os.path.join(output_dir, 'summaries')
summary_callback = tf.keras.callbacks.TensorBoard(summary_dir, profile_batch=0)
time_history_callback = keras_utils.TimeHistory(
    batch_size=args.train_batch_size,
    log_steps=args.time_history_log_steps,
    logdir=summary_dir)
custom_callbacks = [summary_callback, time_history_callback]
if args.save_model:
    #logger.info('Using save_model option...')
    checkpoint_path = os.path.join(output_dir, 'checkpoint')
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
    custom_callbacks.append(checkpoint_callback)
if args.early_stopping_epochs > 0:
    #logger.info(f'Using early stopping of after {args.early_stopping_epochs} epochs of val_loss not decreasing')
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=args.early_stopping_epochs, monitor='val_loss')
    custom_callbacks.append(early_stopping_callback)

In [64]:
def get_dataset_fn(input_file_pattern, max_seq_length, global_batch_size, is_training=True):
  """Gets a closure to create a dataset."""
  def _dataset_fn(ctx=None):
    """Returns tf.data.Dataset for distributed BERT pretraining."""
    batch_size = ctx.get_per_replica_batch_size(
        global_batch_size) if ctx else global_batch_size
    dataset = input_pipeline.create_classifier_dataset(
        input_file_pattern,
        max_seq_length,
        batch_size,
        is_training=is_training,
        input_pipeline_context=ctx)
    return dataset

  return _dataset_fn

In [71]:
# Generate dataset_fn
train_input_fn = get_dataset_fn(
    data_dir+'/tfrecords/train.tfrecords',
    max_seq_length,
    args.train_batch_size,
    is_training=True)
eval_input_fn = get_dataset_fn(
    data_dir+'/tfrecords/dev.tfrecords',
    max_seq_length,
    args.eval_batch_size,
    is_training=False)


In [72]:
# Add mertrics callback to calculate performance metrics at the end of epoch
performance_metrics_callback = Metrics(
        eval_input_fn,
        label_mapping,
        os.path.join(summary_dir, 'metrics'),
        eval_steps,
        args.eval_batch_size,
        validation_freq)
custom_callbacks.append(performance_metrics_callback)

In [73]:
# Run keras fit
time_start = time.time()
#logger.info('Run training...')
history = classifier_model.fit(
    x=train_input_fn(),
    validation_data=eval_input_fn(),
    steps_per_epoch=steps_per_epoch,
    epochs=args.num_epochs,
    validation_steps=eval_steps,
    validation_freq=validation_freq,
    callbacks=custom_callbacks,
    verbose=1)
time_end = time.time()
training_time_min = (time_end-time_start)/60
#logger.info(f'Finished training after {training_time_min:.1f} min')


255/255 [==============================] - ETA: 0s - loss: 0.7549 - accuracy: 0.6740
Epoch 00001: saving model to drive/MyDrive/ct-bert/data/finetune/run_2021-11-24_07-56-06_744272_test_run/crowdbreaks/checkpoint


NotFoundError: ignored

In [74]:
# Write training log
all_scores = performance_metrics_callback.scores
all_predictions = performance_metrics_callback.predictions
if len(all_scores) > 0:
    final_scores = all_scores[-1]
    #logger.info(f'Final eval scores: {final_scores}')
else:
    final_scores = {}
full_history = history.history
if len(full_history) > 0:
    final_val_loss = full_history['val_loss'][-1]
    final_loss = full_history['loss'][-1]
    #logger.info(f'Final training loss: {final_loss:.2f}, Final validation loss: {final_val_loss:.2f}')
else:
    final_val_loss = None
    final_loss = None
data = {
        'created_at': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'run_name': run_name,
        'final_loss': final_loss,
        'final_val_loss': final_val_loss,
        'max_seq_length': max_seq_length,
        'num_train_steps': steps_per_epoch * args.num_epochs,
        'eval_steps': eval_steps,
        'steps_per_epoch': steps_per_epoch,
        'training_time_min': training_time_min,
        'data_dir': data_dir,
        'output_dir': output_dir,
        'all_scores': all_scores,
        'all_predictions': all_predictions,
        'num_labels': num_labels,
        'label_mapping': label_mapping,
        **full_history,
        **final_scores,
        'args':args}

NameError: ignored

In [ ]:
os.path.join(output_dir, run_name, 'run_logs.json')

'drive/MyDrive/vax_bert_output/run_2021-11-19_14-17-34_470901/_run_logs.json'

In [ ]:
# Write run_log
f_path_training_log = os.path.join(output_dir, run_name, '_run_logs.json')
#logger.info(f'Writing training log to {f_path_training_log}...')
save_to_json(data, f_path_training_log)
# Write bert config
model_config.id2label = label_mapping
model_config.label2id = {v:k for k, v in label_mapping.items()}
model_config.max_seq_length = max_seq_length
model_config.num_labels = num_labels
f_path_bert_config = os.path.join(output_dir,  run_name, 'bert_config.json')
#logger.info(f'Writing BERT config to {f_path_bert_config}...')
save_to_json(model_config.to_dict(), f_path_bert_config)